## Lab3: Molecular Dynamics

In this lab, we will prepare and run the necessary steps to perform molecular dynamics (MD) simulations of the cannabinoid 2 (CB2) receptor-ligand complex. 

The lab includes a series of steps to interact with the GROMACS program, as well as with a few Python scripts included within the Python package PyMemDyn. Normally, these steps are all run at once, but here we run them individually to better understand which steps are needed to successfully run MD on a membrane protein.

I recommend to read up on PyMemDyn and related webserver if the following papers: \
https://portlandpress.com/biochemsoctrans/article-abstract/41/1/205/68208/Characterization-of-the-dynamic-events-of-GPCRs-by \
https://academic.oup.com/nar/article/44/W1/W455/2499371?login=true

Below, you will find the relevant code blocks that you can execute by using the play button on top, or by typing Shift+Enter. You will use PyMOL to visualize the process of creating an MD simulation, and the MD simulation itself.

If you want to know more about what the code is doing, please ask one of the teachers!

### Load PyMemDyn

In [ ]:
# Import python native functions
import os
import sys

# Add PyMemDyn modules
sys.path.insert(0, '../')

import settings
from pymemdyn import Run

### Input

We copy in the protein and ligand input files as these have already been pre-processed before the lab.

In [ ]:
if not os.getcwd().endswith('rundir'):
    os.chdir('rundir')

In [ ]:
# copy input files to working directory
!cp ../data/protein.pdb .
!cp ../data/E3R.pdb .
!cp ../data/E3R.itp .
!cp ../data/E3R.ff .

We use the crystal structure 6KPF (https://www.rcsb.org/structure/6KPF) as a starting point. By opening the protein.pdb file in PyMOL, you can see that the structure has been modified compared to the original crystal structure file on the RCSB PDB. 

Due to limitations in crystalizing highly-flexible regions of a protein, some sub-structures are not present within the crystal structure. Therefore, we also connect loops which were disconnected, and check if all amino acid side-chains are present. Also, the G protein was removed from the complex.


For the ligand input, we pre-generated the OPLS forcefield parameters using the LigParGen webserver (http://zarbi.chem.yale.edu/ligpargen/).



### Running PyMemDyn

First, we instantiate the Run class of PyMemDyn, this top level object takes run time variables and stores them in an object so we can use them in the next steps

In [ ]:
run = Run(own_dir = os.getcwd(),
          repo_dir = settings.TEMPLATES_DIR,
          pdb = 'protein.pdb',
          ligand = 'E3R',
          alosteric = None,
          waters = None,
          ions = None,
          cho = None,
          ligpargen = None,
          library = None,
          restraint = 'bw',
          queue = "",
          debug = True
)

The command bellow returns a list of each of the steps run by the PyMemDyn.
We will use these keywords in the following command blocks to run the individual steps
in the program.
Note that PyMemDyn can also run everything in one go (this was the intention of the 
program), but for teaching purposes it makes more sense to carefully look at each individual step.

In [ ]:
run.moldyn_notebook_info()

You can use the code block below to run each of the commands individually, there will be a short description of each of the
steps printed out. Some commands are running via GROMACS (indicated in the header) other commands are actual Python
scripts part of the PyMemDyn package.

Sometimes the help function mentions a filetype (.top or .itp for instance) if you don't know what these files are
you can find more information here: \
https://manual.gromacs.org/documentation/5.1.1/user-guide/file-formats.html


#### Setup Protein-Ligand Complex

In [ ]:
run.moldyn_notebook_run("pdb2gmx")

In [ ]:
run.moldyn_notebook_run("set_itp")

In [ ]:
run.moldyn_notebook_run("concat")

In [ ]:
run.moldyn_notebook_run("editconf")

In [ ]:
run.moldyn_notebook_run("set_protein_size")

In [ ]:
run.moldyn_notebook_run("editconf2")

In [ ]:
run.moldyn_notebook_run("set_protein_size2")

Now, the protein-ligand complex is essembled and positioned for insertion within the membrane. \
Open the proteinopls.pdb file in PyMOL to see what the complex looks like.

#### Embed Complex within Membrane

In [ ]:
run.moldyn_notebook_run("set_popc")

In [ ]:
run.moldyn_notebook_run("editconf3")

In [ ]:
run.moldyn_notebook_run("make_ndx_lig")

In [ ]:
run.moldyn_notebook_run("genrestr_lig")

In [ ]:
run.moldyn_notebook_run("editconf4")

In [ ]:
run.moldyn_notebook_run("make_topol")

In [ ]:
run.moldyn_notebook_run("editconf5")

In [ ]:
run.moldyn_notebook_run("solvate")

Now, the protein-ligand complex is embedded within the lipid bilayer. \
Open the protpopc.pdb file in PyMOL to see what the system now looks like.

#### Solvate Protein-Membrane System with Waters

In [ ]:
run.moldyn_notebook_run("set_water")

In [ ]:
run.moldyn_notebook_run("editconf6")

In [ ]:
run.moldyn_notebook_run("editconf7")

In [ ]:
run.moldyn_notebook_run("solvate2")

Now, the protein-membrane system is solvated with waters. \
Open the tmp.pdb file in PyMOL to see what the complex now looks like.

#### Prepare MD (1)

In [ ]:
run.moldyn_notebook_run("count_lipids")

In [ ]:
run.moldyn_notebook_run("make_topol2")

In [ ]:
run.moldyn_notebook_run("make_topol_lipids")

In [ ]:
run.moldyn_notebook_run("make_ffoplsaanb")

In [ ]:
run.moldyn_notebook_run("set_grompp")

In [ ]:
run.moldyn_notebook_run("set_chains")

In [ ]:
run.moldyn_notebook_run("make_ndx")

In [ ]:
run.moldyn_notebook_run("grompp")

In [ ]:
run.moldyn_notebook_run("trjconv")

#### Add Ions to System


In [ ]:
run.moldyn_notebook_run("get_charge")

In [ ]:
run.moldyn_notebook_run("genion")

Now, the complex is made neutral in charge by adding ions. \
Open the output.pdb file in PyMOL to see what the system now looks like.

#### Prepare MD (2)

In [ ]:
run.moldyn_notebook_run("grompp2")

In [ ]:
run.moldyn_notebook_run("trjconv2")

In [ ]:
run.moldyn_notebook_run("grompp3")

In [ ]:
run.moldyn_notebook_run("trjconv3")

In [ ]:
run.moldyn_notebook_run("clean_pdb")

The final system for MD simulation has now been created. The system has now been converted into an hexagonal shape. \
Open the hexagon.pdb file in PyMOL to see what the system now looks like.

### MD Simulation

**After** you have completed all steps above, you would normally start running the MD simulation within PyMemDyn. However it takes a few hours to complete on a pretty large cluster,
so we have already done it for you.
The output from a PyMemDyn run can be found in the finalOutput folder, which we copy in here now.

In [ ]:
!cp -r ../data/finalOutput .

Typically, it's useful to analyze a few properties, like temperature and pressure, but also RMSD and RMSF (if you don't know what those are please Google it or ask a TA :) ) are good indicators for the stability of your simulation. We can use a basic plotting program called Grace to work with GROMACS output files, e.g.: to check the temperature during your simulation run:

In [ ]:
!xmgrace finalOutput/reports/temp.xvg

In [ ]:
!xmgrace finalOutput/reports/temp2.xvg

In [ ]:
!xmgrace finalOutput/reports/rmsd-all-atom-vs-start.xvg

Plot the other properties and RMSD/RMSF in the /reports folder as well (all the other .xvg files), and save them in a file. Note that the properties are seperated into two files, each for a specific part of the simulation. Write a short description for each of the figures. Discuss what you are seeing with your fellow students and/or TAs. Note that you need to close the xmgrace window that opened everytime you want to load a new figure, or you need to add a new code block.

Next, we can have a look at the trajectory. Open PyMOL and change to the finalOutput directory. There, you can load a PyMOL script (.pml) generated by PyMemDyn, by typing @load_gpcr.pml

- Can you see when the restraints are released in the simulation? 
- Can you link that to events described in the PyMemDyn and GPCR-ModSim papers?
- Do you think the ligand looks stable?
- What could you do to further assess ligand stability?
- Do we know anything about the binding affinity now?

Thanks for participating in this last computerlab, as always please ask your TA any questions if something is unclear!